In [1]:
# Bibliotecas utilizadas

import pandas as pd
import os
import pyodbc
import numpy as np
import re
import shutil
import matplotlib.pyplot as plt

In [2]:
# Conexão com banco de dados

sqlgdncli = pyodbc.connect("DRIVER={SQL Server};SERVER=SQLGDNCLI;DATABASE=WORKSPACE;Trusted_Connection=yes")

In [3]:
# Função para ler todos os arquivos

def ReadFiles(path, name_file, separador):
    
    file = f'{path}{name_file}'

    names = [
        'CPF',
        'NOME',
        'NASC',
        'NOME_MAE',
        'SEXO',
        'SIGNO',
        'RENDA',
        'ESTADOCIVIL',
        'CBO',
        'DESCRICAO_CBO',
        'ESCOLARIDADE',
        'DESCRICAO_ESCOLARIDADE',
        'TIPO',
        'TITULO',
        'LOGRADOURO',
        'NUMERO',
        'COMPLEMENTO',
        'BAIRRO',
        'CIDADE',
        'UF',
        'CEP',
        'DDD1',
        'FONE1',
        'TIPO1',
        'DDD2',
        'FONE2',
        'TIPO2',
        'DDD3',
        'FONE3',
        'TIPO3',
        'DDD4',
        'FONE4',
        'TIPO4',
        'DDD5',
        'FONE5',
        'TIPO5',
        'DDD6',
        'FONE6',
        'TIPO6',
        'DDD7',
        'FONE7',
        'TIPO7',
        'DDD8',
        'FONE8',
        'TIPO8',
        'EMAIL_1',
        'EMAIL_2',
        'EMAIL_3',
    ]

    df = pd.read_csv(file, sep=separador, names = names, header=None, skiprows = 1, index_col = False, encoding='ISO-8859-1')

    # Substituir NaN por 'Desconhecido'
    df['ESTADOCIVIL'].fillna('Desconhecido', inplace=True)

    # Utilizar expressão regular para encontrar o ano
    padrao = r'\\(\d{4})\\'
    correspondencia = re.search(padrao, path)
    ano = correspondencia.group(1)
        
    df['CAMINHO'] = file
    df['ARQUIVO'] = name_file
    df['ANO'] = ano

    return df

In [4]:
def ConcatenaArquivos(path, separador):

    global df_final
    
    # Lista com o nome de cada arquivo do Path
    name_file = os.listdir(path)

    # estrutura para concatenar cada arquivo em um DF unico
    for i, y in enumerate(name_file):

        df = ReadFiles(path, y, separador)

        if i == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])

In [5]:
# Inserir Dataframe no SQL Server:

def InserirDadosBD():

        cursor = sqlgdncli.cursor()

        # Configurar a quantidade de linhas por lote
        tamanho_do_lote = 500
        cont = 0
        params_list = []
        global df_final

        df_final = df_final.astype(str)

        # DataFrame df_final
        for index, row in df_final.iterrows():
                cont = cont + 1

                # Adicionar os parâmetros à lista
                params_list.append((
                        row.CPF, row.NOME, row.NASC, row.NOME_MAE, row.SEXO, row.SIGNO, row.RENDA, row.ESTADOCIVIL, row.CBO, row.DESCRICAO_CBO,
                        row.ESCOLARIDADE, row.DESCRICAO_ESCOLARIDADE, row.TIPO, row.TITULO, row.LOGRADOURO, row.NUMERO, row.COMPLEMENTO, row.BAIRRO,
                        row.CIDADE, row.UF, row.CEP, row.DDD1, row.FONE1, row.TIPO1, row.DDD2, row.FONE2, row.TIPO2, row.DDD3, row.FONE3, row.TIPO3,
                        row.DDD4, row.FONE4, row.TIPO4, row.DDD5, row.FONE5, row.TIPO5, row.DDD6, row.FONE6, row.TIPO6, row.DDD7, row.FONE7, row.TIPO7,
                        row.DDD8, row.FONE8, row.TIPO8, row.EMAIL_1, row.EMAIL_2, row.EMAIL_3, row.CAMINHO, row.ARQUIVO, row.ANO
                ))

                # Inserir em lote
                if cont >= tamanho_do_lote:
                        cursor.executemany('''
                        INSERT INTO TE_DADOS_ENRIQ_TELEFONES_ALERTAKI 
                        (CPF, NOME, NASC, NOME_MAE, SEXO, SIGNO, RENDA, ESTADOCIVIL, CBO, DESCRICAO_CBO, ESCOLARIDADE, DESCRICAO_ESCOLARIDADE,
                        TIPO, TITULO, LOGRADOURO, NUMERO, COMPLEMENTO, BAIRRO, CIDADE, UF, CEP, DDD1, FONE1, TIPO1, DDD2, FONE2, TIPO2, DDD3, FONE3,
                        TIPO3, DDD4, FONE4, TIPO4, DDD5, FONE5, TIPO5, DDD6, FONE6, TIPO6, DDD7, FONE7, TIPO7, DDD8, FONE8, TIPO8, EMAIL_1, EMAIL_2,
                        EMAIL_3, CAMINHO, ARQUIVO, ANO) 
                        VALUES 
                        (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                        ''', params_list)

                        # Commit
                        cursor.commit()

                        # Limpar a lista de parâmetros
                        params_list = []
                        cont = 0

        # Inserir quaisquer linhas remanescentes
        if cont > 0:
                cursor.executemany('''
                        INSERT INTO TE_DADOS_ENRIQ_TELEFONES_ALERTAKI 
                        (CPF, NOME, NASC, NOME_MAE, SEXO, SIGNO, RENDA, ESTADOCIVIL, CBO, DESCRICAO_CBO, ESCOLARIDADE, DESCRICAO_ESCOLARIDADE,
                        TIPO, TITULO, LOGRADOURO, NUMERO, COMPLEMENTO, BAIRRO, CIDADE, UF, CEP, DDD1, FONE1, TIPO1, DDD2, FONE2, TIPO2, DDD3, FONE3,
                        TIPO3, DDD4, FONE4, TIPO4, DDD5, FONE5, TIPO5, DDD6, FONE6, TIPO6, DDD7, FONE7, TIPO7, DDD8, FONE8, TIPO8, EMAIL_1, EMAIL_2,
                        EMAIL_3, CAMINHO, ARQUIVO, ANO) 
                        VALUES 
                        (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''', params_list)

                # Commit final
                cursor.commit()

        cursor.close()


In [6]:
# Inserir os dados em um CSV

def InsereCSV():
    
    cursor = sqlgdncli.cursor()

    # Sua consulta SQL
    sql_query = '''
        SELECT DISTINCT A.NUM_PES, B.ESTADOCIVIL
        FROM [MERCANTIL\\b042304].[TE_ENRIQUECIMENTO_ESTADO_CIVIL] A
        INNER JOIN (
            SELECT DISTINCT NUM_PES, ESTADOCIVIL
            FROM [MERCANTIL\\b042304].[TI_BENEF_FINAL]
            WHERE ESTADOCIVIL IN ('CASADO')

            UNION 

            SELECT DISTINCT NUM_PES, ESTADOCIVIL
            FROM [MERCANTIL\\b042304].[TI_BENEF_FINAL]
            WHERE ESTADOCIVIL IN ('UNIAO ESTAVEL')
            AND NUM_PES NOT IN (
                SELECT NUM_PES
                FROM [MERCANTIL\\b042304].[TI_BENEF_FINAL]
                WHERE ESTADOCIVIL IN ('CASADO')
                )

            UNION

            SELECT DISTINCT B.NUM_PES, A.ESTADOCIVIL
            FROM TE_DADOS_ENRIQ_TELEFONES_ALERTAKI A
            INNER JOIN [SHARESPACE].[MERCANTIL\Y041326].[TC_PERFIL_CLIENTE] B
            ON A.CPF = B.CPF
            WHERE ESTADOCIVIL = 'CASADO'
            AND NUM_PES NOT IN (
                SELECT DISTINCT NUM_PES
                FROM [MERCANTIL\\b042304].[TI_BENEF_FINAL]
                WHERE ESTADOCIVIL IN ('UNIAO ESTAVEL', 'CASADO')
                )
            ) B
        ON A.NUM_PES = B.NUM_PES
    '''

    # Executar a consulta e ler o resultado em um DataFrame
    df_resultado = pd.read_sql_query(sql_query, cursor)

    # Salvar o DataFrame em um arquivo CSV
    df_resultado.to_csv('resultado_query.csv', index=False)

    # Fechar a conexão
    cursor.close()

In [7]:
# Variaveis globais
controle = 0

# Separador '|'
path_1 = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\EnriquecimentoEstadoCivil\\Arquivos\\2021\\'
path_2 = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\EnriquecimentoEstadoCivil\\Arquivos\\2022\\'
path_3 = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\EnriquecimentoEstadoCivil\\Arquivos\\2023\\'

# Separador ';'
path_4 = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\EnriquecimentoEstadoCivil\\FormatosDiferentes\\2023\\'

# Separador ','
path_5 = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\EnriquecimentoEstadoCivil\\FormatosDiferentes\\2022\\'

ConcatenaArquivos(path_1, '|')
InserirDadosBD()
ConcatenaArquivos(path_2, '|')
InserirDadosBD()
ConcatenaArquivos(path_3, '|')
InserirDadosBD()
ConcatenaArquivos(path_4, ';')
InserirDadosBD()
ConcatenaArquivos(path_5, ',')
InserirDadosBD()

InsereCSV()

C:\Users\B042304\AppData\Local\Temp\ipykernel_6760\817137916.py:58: DtypeWarning: Columns (0,8,10,20,21,22,24,25,27,28,30,31,33,34,36,37,39,40,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=separador, names = names, header=None, skiprows = 1, index_col = False, encoding='ISO-8859-1')
C:\Users\B042304\AppData\Local\Temp\ipykernel_6760\817137916.py:58: DtypeWarning: Columns (13,15,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=separador, names = names, header=None, skiprows = 1, index_col = False, encoding='ISO-8859-1')
C:\Users\B042304\AppData\Local\Temp\ipykernel_6760\817137916.py:58: DtypeWarning: Columns (3,7,11,12,13,14,15,16,17,18,19,41,44,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=separador, names = names, header=None, skiprows = 1, index_col = False, encoding='ISO-8859-1')
C:\Users\B042304\App

AttributeError: 'pyodbc.Cursor' object has no attribute 'cursor'

In [ ]:
# Avalia os arquivos com separadores diferentes

avaliar = df_final[df_final['NOME'].isnull()]
avaliar = avaliar['ARQUIVO'].unique()

# Remove da pasta os arquivos com separadores diferentes

caminho_origem = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\LeituraArquivosTelefone\\Arquivos\\2023\\'
caminho_destino = 'I:\\Crescimento\\Temporário\\Rheydne\\Python\\LeituraArquivosTelefone\\'

for i in avaliar:
    caminho = f'{caminho_origem}{i}'
    shutil.move(caminho, caminho_destino)

In [ ]:
# Avaliação Estado Civil

# Contagem de pessoas por estado civil
contagem_estado_civil = df_final['ESTADOCIVIL'].value_counts()

# Gráfico de barras
contagem_estado_civil.plot(kind='bar', color='skyblue', edgecolor='black')

# Calcular porcentagens
porcentagens = (contagem_estado_civil / contagem_estado_civil.sum()) * 100

# Gráfico de barras com porcentagens internas
ax = contagem_estado_civil.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Quantidade de Pessoas por Estado Civil (Incluindo NaN)')
plt.xlabel('Estado Civil')
plt.ylabel('Quantidade')

# Adicionar porcentagens internas nas barras
for i, v in enumerate(contagem_estado_civil):
    ax.text(i, v + 0.1, f'{porcentagens.iloc[i]:.1f}%', ha='center')

plt.show()